In [ ]:
from os import path

from astropy.constants import c
import astropy.coordinates as coord
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('apw-notebook')
%matplotlib inline
from sqlalchemy import func

from comoving_rv.log import logger
from comoving_rv.db import Session, Base, db_connect
from comoving_rv.db.model import (Run, Observation, TGASSource, SimbadInfo, PriorRV,
                                  SpectralLineInfo, SpectralLineMeasurement, RVMeasurement)

In [ ]:
base_path = '/Volumes/ProjectData/gaia-comoving-followup/'
db_path = path.join(base_path, 'db.sqlite')
engine = db_connect(db_path)
session = Session()

In [ ]:
group_ids = np.array([x[0] 
                      for x in session.query(Observation.group_id).distinct().all() 
                      if x[0] is not None and x[0] > 0 and x[0] != 10])

In [ ]:
raw_dv = []
corr_dv = []
seps = []
errs = []

base_q = session.query(Observation).join(RVMeasurement).filter(RVMeasurement.rv != None)
for gid in np.unique(group_ids):
    observations = base_q.filter(Observation.group_id == gid).all()
    
    if len(observations) != 2:
        print("skipping group {0}".format(gid))
        continue
        
    obs1, obs2 = observations
    
    corr_dv.append(obs1.rv_measurement.rv - obs2.rv_measurement.rv)
    raw_dv.append((obs1.measurements[0].x0 - obs2.measurements[0].x0) / obs1.measurements[0].x0 * c.to(u.km/u.s))
    seps.append(obs1.tgas_source.skycoord.separation(obs2.tgas_source.skycoord))
    
    rv1_err = obs1.measurements[0].x0_error / obs1.measurements[0].x0 * c.to(u.km/u.s)
    rv2_err = obs1.measurements[0].x0_error / obs1.measurements[0].x0 * c.to(u.km/u.s)
    errs.append(np.sqrt(rv1_err**2 + rv2_err**2))
    
corr_dv = u.Quantity(corr_dv)
raw_dv = u.Quantity(raw_dv)
seps = u.Quantity(seps)
errs = u.Quantity(errs)

In [ ]:
plt.hist(raw_dv, bins='auto');

In [ ]:
plt.hist(corr_dv, bins='auto');

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)

# axes[0].scatter(seps.to(u.degree).value, np.abs(raw_dv))
# axes[1].scatter(seps.to(u.degree).value, np.abs(corr_dv))
axes[0].errorbar(seps.to(u.degree).value, raw_dv.value, yerr=errs.value,
                 linestyle='none', ecolor='#aaaaaa', marker='.')
axes[1].errorbar(seps.to(u.degree).value, corr_dv.value, yerr=errs.value, 
                 linestyle='none', ecolor='#aaaaaa', marker='.')

axes[0].set_xscale('log')
axes[0].set_xlim(1E-1, 2E1)
axes[0].set_ylim(-75, 75)

axes[0].set_xlabel('sky sep [deg]')
axes[0].set_ylabel(r'${{\rm RV}}_1 - {{\rm RV}}_2$ [{0}]'.format((u.km/u.s).to_string('latex_inline')))